In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

In [2]:
data = pd.read_csv('/Users/shiwei/Desktop/研究论文/一带一路ERGM数据分析/all_BeltandRoad_tweets.csv')

/var/folders/wq/891pj93x7r99n97_2h5wxzzw0000gn/T/ipykernel_7108/1205769208.py:1: DtypeWarning: Columns (3,11,19,39,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/shiwei/Desktop/研究论文/一带一路ERGM数据分析/all_BeltandRoad_tweets.csv')


In [3]:
data_user = pd.read_csv('/Users/shiwei/Desktop/研究论文/一带一路回音室/1551节点属性.csv')

In [6]:
data = data[['author.username', 'text']]
data

,author.username,text
0,LojisticUSA,The new silk road. What are your thoughts on t...
1,A1SocialNews,The new Silk Road โ€“ railways from China to W...
2,realbadceo,The New Silk Road: China-Europe Railways http:...
3,jyoungtech,One day I'm going to check this out. The New ...
4,byosiyana,Check out the latest edition of โ€JICA Worldโ€...
...,...,...
1248541,wattscentral,.@Matt_Schrader_ and @JMichaelCole1 discuss Ch...
1248542,HongPong,"this says ""belt and road"" v1.0 is over . hadn'..."
1248543,FazdillahReza,https://t.co/wVUdQPQF5k\n\n“If the government ...
1248544,YerkoIlijic,.@Matt_Schrader_ and @JMichaelCole1 discuss Ch...


In [5]:
data_user

,username,neg,neu,pos,compound,sumpopular
0,760462590,0.000629,0.809444,0.189928,0.387547,0.000216
1,_JakubJanda,0.057431,0.868031,0.074523,0.086272,0.281920
2,_rjardon,0.020743,0.917457,0.061786,0.098417,0.063966
3,_XploreBD,0.016778,0.916952,0.066270,0.134112,0.004543
4,18AKGOYAL,0.053771,0.877892,0.068325,0.057465,0.098551
...,...,...,...,...,...,...
1546,ZSOpinion,0.044376,0.876295,0.079324,0.147796,0.133479
1547,zulfiqarspeaks,0.013636,0.815803,0.170561,0.429921,0.215453
1548,zuoluogu,0.044508,0.885831,0.069631,0.141383,0.203196
1549,zupazwojtusia,0.084549,0.848882,0.066569,-0.058176,0.254829


In [7]:
data = data[data['author.username'].isin(data_user['username'])]

,author.username,text
6,Kostian_V,Parag Khanna ( @paragkhanna ) - The new Silk R...
27,SouthCaucasus,The new Silk Road. Germany to China rail route...
33,JamestownTweets,Chinaโ€s Latest Piece of the New Silk Road htt...
38,Kostian_V,The Jamestown Foundation: #China โ€s Latest Pi...
69,Edourdoo,The new Silk Road is made of ironโ€”and stretc...
...,...,...
1248525,ProfJonTaylor,China Hasn’t Given Up on the Belt and Road -- ...
1248526,NamJustus,China Hasn’t Given Up on the Belt and Road htt...
1248529,KleptocracyIntv,Policymakers in developed and emerging economi...
1248530,Brooklynmonk,Beyond the Belt and Road: China’s Global Econo...


In [8]:
# 重置索引以解决索引问题
data.reset_index(drop=True, inplace=True)
data

,author.username,text
0,Kostian_V,Parag Khanna ( @paragkhanna ) - The new Silk R...
1,SouthCaucasus,The new Silk Road. Germany to China rail route...
2,JamestownTweets,Chinaโ€s Latest Piece of the New Silk Road htt...
3,Kostian_V,The Jamestown Foundation: #China โ€s Latest Pi...
4,Edourdoo,The new Silk Road is made of ironโ€”and stretc...
...,...,...
224499,ProfJonTaylor,China Hasn’t Given Up on the Belt and Road -- ...
224500,NamJustus,China Hasn’t Given Up on the Belt and Road htt...
224501,KleptocracyIntv,Policymakers in developed and emerging economi...
224502,Brooklynmonk,Beyond the Belt and Road: China’s Global Econo...


In [9]:
import nltk
import re

In [10]:
# 导入nltk的分词工具word_tokenize和nltk的停用词表stopwords，需要自行下载哦
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopwords_list = stopwords.words("english")

In [11]:
# 定义一个函数，处理Twitter文本中的特殊符号：@xxx、&xxx、网址、多余的空格，最后只保留数字和字母
def clean_specialtext(tweet):
    tweet = re.sub(r"@([^\s:]+)", " ", tweet)  # 去除@提及内容
    tweet = re.sub(r"&([^\s:]+)", " ", tweet)  # 去除&及之后的内容
    tweet = re.sub(r"[a-zA-z]+://[^\s]*", " ", tweet)  # 去除网址
    tweet = re.sub(r"[^0-9A-Za-z]", " ", tweet)  # 仅保留数字及字母
    tweet = re.sub(r"(^\s*)|(\s*$)", "", tweet)  # 去除字符串前后空格
    # tweet = re.sub(r'[^0-9A-Za-z]', ' ', tweet)
    return tweet

In [12]:
data["text"] = data["text"].apply(lambda x: clean_specialtext(x))

/var/folders/wq/891pj93x7r99n97_2h5wxzzw0000gn/T/ipykernel_7108/571408044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data["text"].apply(lambda x: clean_specialtext(x))


In [13]:
data

,author.username,text
0,Kostian_V,Parag Khanna The new Silk Road is made...
1,SouthCaucasus,The new Silk Road Germany to China rail route...
2,JamestownTweets,China s Latest Piece of the New Silk Road
3,Kostian_V,The Jamestown Foundation China s Latest Pi...
4,Edourdoo,The new Silk Road is made of iron and stretc...
...,...,...
224499,ProfJonTaylor,China Hasn t Given Up on the Belt and Road ...
224500,NamJustus,China Hasn t Given Up on the Belt and Road via
224501,KleptocracyIntv,Policymakers in developed and emerging economi...
224502,Brooklynmonk,Beyond the Belt and Road China s Global Econo...


In [14]:
#导入nltk的词性标注工具pos_tag、词性属性工具wordnet和词形还原工具WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

In [15]:
# 这一步是为了更精确词形还原，对词性进行标注，精确的词性对词形还原很有必要
# 参考https://stackoverflow.com/questions/61982023/using-wordnetlemmatizer-lemmatize-with-pos-tags-throws-keyerror
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return None

In [16]:
# 定义去除停用词和词形还原函数，每一步都是有顺序的，不能乱

def clean_stopwords_Lemmatization(tweet):
    tweet = word_tokenize(tweet)
    # print(tweet)
    # print("*"*100)
    tweet = [word for word in tweet if word not in stopwords_list]  # 去除停用词
    tweet = pos_tag(tweet)  # 词性标注
    # 词性标注好之后，列表里变成了（词，词性）的元组形式，word[0]是词的位置，对word[0]进行词形还原，依据是上面的函数和word[1]
    tweet = [
        lemmatizer.lemmatize(word[0], pos=get_wordnet_pos(word[1]))
        if get_wordnet_pos(word[1])
        else word[0]
        for word in tweet
    ]
    tweet = [word for word in tweet if len(word) > 2]  # 丢弃长度小于等于2的字符串
    tweet = " ".join(tweet)  # 将tweet列表中的字符串重新拼接起来，用空格分隔
    # print(tweet)
    # print("#"*100)
    return tweet

In [17]:
data['text'] = data['text'].apply(lambda x: clean_stopwords_Lemmatization(x))

/var/folders/wq/891pj93x7r99n97_2h5wxzzw0000gn/T/ipykernel_7108/1612954280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: clean_stopwords_Lemmatization(x))


In [18]:
data

,author.username,text
0,Kostian_V,Parag Khanna The new Silk Road make iron stret...
1,SouthCaucasus,The new Silk Road Germany China rail route rev...
2,JamestownTweets,China Latest Piece New Silk Road
3,Kostian_V,The Jamestown Foundation China Latest Piece Ne...
4,Edourdoo,The new Silk Road make iron stretch Scotland S...
...,...,...
224499,ProfJonTaylor,China Hasn Given Belt Road
224500,NamJustus,China Hasn Given Belt Road via
224501,KleptocracyIntv,Policymakers develop emerge economy need work ...
224502,Brooklynmonk,Beyond Belt Road China Global Economic Expansi...


In [20]:
data.to_csv('/Users/shiwei/Desktop/研究论文/一带一路回音室/1551用户清洗完毕的帖子224504.csv', encoding = 'utf8')

In [21]:
# 加载模型（如果需要）
model = Word2Vec.load("word2vec.model")

In [22]:
# 查找某个单词的词向量
word_vector = model.wv['xinjiang']
print("词向量 for 'xinjiang':")
print(word_vector)

词向量 for 'xinjiang':
[ 4.35736030e-01  7.62851954e-01 -6.90107569e-02 -4.16154861e-02
 -1.30970752e+00  1.06725156e-01  4.52926680e-02  1.95536524e-01
 -1.00060403e+00 -6.60187542e-01 -1.05257964e+00 -1.01040363e+00
  8.09402585e-01  1.39764023e+00  6.58127293e-02 -9.39076185e-01
 -8.02247226e-01 -4.76894289e-01  3.13913971e-01  1.01920319e+00
  3.16260338e-01 -1.54354823e+00 -3.36530775e-01  9.11720395e-01
 -5.04368424e-01 -4.84725803e-01 -9.37605083e-01  1.29498816e+00
  2.17967343e+00 -3.60700399e-01  1.50325143e+00  1.13270915e+00
  4.73999560e-01  2.16967440e+00  1.19049698e-01 -3.37016910e-01
 -9.37347054e-01  3.89312836e-03  4.22661811e-01  4.80155557e-01
 -4.74220246e-01 -4.17478323e-01  9.40312624e-01 -6.87745392e-01
 -9.07693863e-01 -4.57184136e-01  8.60631168e-01  3.45189035e-01
 -2.55526543e-01 -3.08912873e-01 -1.06212437e+00  1.96319029e-01
 -1.04435742e+00 -1.76426917e-01 -5.05139604e-02  6.30788386e-01
 -9.80269015e-01  1.05175197e+00 -7.02683687e-01 -2.86898315e-01
 -9.9

In [23]:
# 词向量加权平均，获取每一条推文的向量表示
def get_post_vector(post, model):
    # 分词帖子文本
    words = post.split()  # 假设帖子已分词
    word_vectors = [model.wv[word] for word in words if word in model.wv]

    if len(word_vectors) > 0:
        post_vector = np.mean(word_vectors, axis=0)  # 求均值
        return post_vector
    else:
        return np.zeros(model.vector_size)  # 如果帖子中没有词汇在模型中

In [24]:
# 假设你的DataFrame中有一个名为df的数据框，包含帖子文本在"text"列中
data['post_vector'] = data['text'].astype(str).apply(lambda x: get_post_vector(x, model))

/var/folders/wq/891pj93x7r99n97_2h5wxzzw0000gn/T/ipykernel_7108/108252988.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_vector'] = data['text'].astype(str).apply(lambda x: get_post_vector(x, model))


In [25]:
data

,author.username,text,post_vector
0,Kostian_V,Parag Khanna The new Silk Road make iron stret...,"[0.53127563, 0.2690652, -0.29646966, -0.026611..."
1,SouthCaucasus,The new Silk Road Germany China rail route rev...,"[-0.16926704, 0.5385657, -0.27217385, 0.252092..."
2,JamestownTweets,China Latest Piece New Silk Road,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Kostian_V,The Jamestown Foundation China Latest Piece Ne...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Edourdoo,The new Silk Road make iron stretch Scotland S...,"[0.42055026, 0.45105895, -0.06952037, 0.059240..."
...,...,...,...
224499,ProfJonTaylor,China Hasn Given Belt Road,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
224500,NamJustus,China Hasn Given Belt Road via,"[-0.022351308, 1.179034, 0.8382768, 0.40264854..."
224501,KleptocracyIntv,Policymakers develop emerge economy need work ...,"[0.033990294, 0.12420395, 0.009195833, -0.1274..."
224502,Brooklynmonk,Beyond Belt Road China Global Economic Expansi...,"[-0.1756794, 0.67931175, 0.12632239, 0.1299322..."


In [26]:
# 创建一个新数据框emotion_vectors，用于存储每个情感层的平均向量

user_vectors = pd.DataFrame(columns=["author.username", "avg_post_vector"])

# 遍历每个情感层
for user, group in data.groupby("author.username"):
    # 计算每个情感层的平均帖子向量
    avg_vector = group["post_vector"].mean()
    
    # 添加到结果数据框
    user_vectors = pd.concat([user_vectors, pd.DataFrame({"author.username": [user], "avg_post_vector": [avg_vector]})], ignore_index=True)

# 输出每个情感层的平均帖子向量
user_vectors

,author.username,avg_post_vector
0,18AKGOYAL,"[0.1383323720712439, 0.263947430106037, 0.2543..."
1,1f8PVxouToaU0R9,"[0.21446763148894724, 0.038286555218636185, -0..."
2,1ironbark1,"[0.17735657224226603, 0.1386213076592826, 0.06..."
3,2050_China,"[0.01799385559429685, 0.1539047448144805, 0.12..."
4,28481k,"[0.16718100265179778, 0.040656950142161516, 0...."
...,...,...
1546,zohrathought,"[0.1920874840350624, -0.0751475294748547, 0.01..."
1547,zulfiqarspeaks,"[0.03285864366669998, 0.19066180168584018, 0.1..."
1548,zuoluogu,"[0.12485479563474655, 0.015074887026387911, 0...."
1549,zupazwojtusia,"[0.16524270936554553, 0.1478136996875572, 0.05..."


In [27]:
user_vectors.to_csv('/Users/shiwei/Desktop/研究论文/一带一路回音室/1551每一个用户向量.csv')